# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID-19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD-COVID-19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://covid19.ibge.gov.br/pnad-covid/

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

In [1]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import pandas as pd
import plotly.express as px

### Conectando ao MySQL

In [3]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

### Criando sessão Spark

In [4]:
findspark.init()

In [5]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/11/02 00:12:34 WARN Utils: Your hostname, platero-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.15.109 instead (on interface wlp2s0)
23/11/02 00:12:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 00:12:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/02 00:12:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

### Importanto dados tratados da view do arquivo CSV

In [7]:
df = spark.read.csv('dados/dados_exportados/dados_uteis/dados_uteis_gz/2023-11-01_pnad_covid_view.csv.gz', sep=',', inferSchema=True, header=True)
df.show(5, truncate=False)

23/11/02 00:13:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+--------------+------------------+-----+------+--------+------------------------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+-----------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|data      |uf            |situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_priva

### Análise exploratória de características clínicas

In [8]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### **<p style='color:gray'> Estatísticas descritivas de pacientes sintomáticos e assintomáticos</p>**

**Qual é a porcetagem de pacientes sintomáticos e assintomáticos?**

In [9]:
df_testes_positivos_sintomas_covid = spark.sql(
    '''
        SELECT 
            CASE
            WHEN sintoma_covid = 'Sim' THEN 'Sintomáticos'
            ELSE 'Assintomáticos' 
            END AS sintoma_covid,
            count(resultado_teste) AS qtd_testes_positivos
        FROM df_temp
        WHERE resultado_teste = 'Positivo'
        GROUP BY sintoma_covid
    '''
).toPandas()

fig = px.pie(
    data_frame=df_testes_positivos_sintomas_covid.sort_values('qtd_testes_positivos', ascending=False),
    values = 'qtd_testes_positivos',
    names = 'sintoma_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'sintoma_covid':'Condição'
    }
)

fig.update_traces(
    pull=[0.1, 0]

)

fig.update_layout(
    title='<b>Porcentagem de infectados sintomáticos e assintomáticos</b>',
    legend_title='Legenda',
    width=800,
    height=600
)

fig.show()

**Qual é porcentagem de infectados por fator de risco?**

In [10]:
df_testes_positivos_fator_risco = spark.sql(
'''
    SELECT
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Diabetes%' THEN 1 ELSE 0 END) /sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 AS Diabetes,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Hipertensao%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Hipertensao,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca respiratoria%' THEN 1 else 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100 as Doenca_respiratoria,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Idoso%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Idoso,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Cancer%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  as Cancer,
        (sum(CASE WHEN descricao_fator_risco_covid LIKE '%Doenca cardiaca%' THEN 1 ELSE 0 END) / sum(CASE WHEN fator_risco_covid = 'Sim' THEN 1 ELSE 0 END)) * 100  AS Doenca_cardiaca
    FROM df_temp
    WHERE resultado_teste = 'Positivo'
'''
).toPandas()

df_testes_positivos_fator_risco = df_testes_positivos_fator_risco.T.reset_index().rename(columns={0: 'percentual_testes_positivos', 'index': 'fator_risco_covid'})
df_testes_positivos_fator_risco.sort_values('percentual_testes_positivos', ascending=False, inplace=True)

fig = px.bar(
    data_frame = df_testes_positivos_fator_risco,
    x = 'fator_risco_covid',
    y = 'percentual_testes_positivos',
    color= 'fator_risco_covid',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'fator_risco_covid': 'Fato de risco'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por fator de risco</b>',
    showlegend=False,
    yaxis_title='%',
    width=800,
    height=600
)

fig.show()

**Qual é porcentagem de infectados por fator de risco?**

In [11]:
df_testes_positivos_tipo_sintoma = spark.sql(
    '''
    SELECT
          (sum(CASE WHEN descricao_sintoma_covid LIKE '%Febre%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END)) * 100 AS Febre,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Tosse%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Tosse,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor de garganta%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_de_garganta,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dificuldade para respirar%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dificuldade_de_respirar,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor de cabeça%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_de_cabeca,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor no peito%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_no_peito,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Nausea%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Nausea,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Fadiga%'THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Fadiga,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor nos olhos%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_nos_olhos,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Perda de olfato ou paladar%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Perda_de_olfato_ou_paladar,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Dor muscular%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Dor_muscular,
          sum(CASE WHEN descricao_sintoma_covid LIKE '%Diarreia%' THEN 1 ELSE 0 END) / sum(CASE WHEN descricao_sintoma_covid IS NOT NULL THEN 1 ELSE 0 END) * 100 AS Diarreia
    FROM df_temp
    WHERE resultado_teste = 'Positivo'
    '''
).toPandas()

df_testes_positivos_tipo_sintoma = df_testes_positivos_tipo_sintoma.T.reset_index().rename(columns={0: 'percentual_testes_positivos', 'index':'sintoma'})
df_testes_positivos_tipo_sintoma.sort_values('percentual_testes_positivos', ascending=False, inplace=True)

fig = px.bar(
    data_frame = df_testes_positivos_tipo_sintoma,
    x = 'sintoma',
    y = 'percentual_testes_positivos',
    color= 'sintoma',
    color_discrete_sequence=['#67000D'],
    labels={
        'percentual_testes_positivos': 'Percentual de testes positivos',
        'sintoma': 'Sintoma'
    }
)

fig.update_layout(
    title='<b>Porcentagem de infectados por tipo de sintoma</b>',
    showlegend=False,
    yaxis_title='%',
    width=800,
    height=600
)

fig.show()

**Qual é a porcentagem de infectado internados?**

In [12]:
df_testes_positivos_internacao = spark.sql(

'''
    select
        count(resultado_teste) AS qtd_testes_positivos,
        CASE
        WHEN questao_internacao = 'Sim' THEN 'Internado'
        WHEN questao_internacao = 'Não' THEN 'Não internado'
        WHEN questao_internacao = 'Não foi atendido' THEN 'Não foi atendido'
        WHEN questao_internacao = 'Ignorado' THEN 'Ignorado'
        ELSE 'Não aplicável' 
        END AS questao_internacao
    FROM df_temp
    WHERE resultado_teste = 'Positivo' AND (questao_internacao = 'Sim' OR questao_internacao = 'Não')
    GROUP BY questao_internacao
'''
).toPandas()

fig = px.pie(
    data_frame=df_testes_positivos_internacao.sort_values('qtd_testes_positivos', ascending=False),
    values='qtd_testes_positivos',
    names='questao_internacao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_internacao':'Condição'
    }
)

fig.update_traces(
    pull=[0, 0.1],
    texttemplate='%{percent:.2%}'
)

fig.update_layout(
    title='<b>Porcentagem de infectados internados</b>',
    legend_title='Legenda',
    width=800,
    height=600
)

fig.show()

**Qual é a porcentagem de infectados que se automedicaram?**

In [13]:
import plotly.graph_objects as go

df_testes_positivos_automedicacao = spark.sql(
    '''
        SELECT  
            questao_remedio_conta_propria AS resposta,
            count(questao_remedio_conta_propria) AS qtd_automedicacao
        FROM df_temp
        WHERE resultado_teste = 'Positivo' AND (questao_remedio_conta_propria = 'Sim' AND questao_remedio_orientacao_medica = 'Não')
        GROUP BY resposta
    '''
).toPandas()

df_testes_positivos_medicacao_orientada = spark.sql(
    '''
        SELECT  
            questao_remedio_orientacao_medica AS resposta,
            count(questao_remedio_orientacao_medica) AS qtd_medicacao_orientada
        FROM df_temp
        WHERE resultado_teste = 'Positivo' AND (questao_remedio_orientacao_medica = 'Sim' AND questao_remedio_conta_propria = 'Não' )
        GROUP BY resposta
    '''
).toPandas()

df_testes_positivos_medicacao_ambos = spark.sql(
    '''
        SELECT  
            questao_remedio_orientacao_medica AS resposta,
            count(questao_remedio_orientacao_medica) AS questao_remedio_ambos
        FROM df_temp
        WHERE resultado_teste = 'Positivo' AND (questao_remedio_orientacao_medica = 'Sim' AND questao_remedio_conta_propria = 'Sim' )
        GROUP BY resposta
    '''
).toPandas()


df_testes_positivos_medicacao = df_testes_positivos_automedicacao.merge(df_testes_positivos_medicacao_orientada, on='resposta')
df_testes_positivos_medicacao = df_testes_positivos_medicacao.merge(df_testes_positivos_medicacao_ambos, on='resposta')


fig = go.Figure(data=[
    go.Bar(name='Automedicados', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.qtd_automedicacao),
    go.Bar(name='Medicados com orientação médica', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.qtd_medicacao_orientada),
    go.Bar(name='Ambos', x=df_testes_positivos_medicacao.resposta, y=df_testes_positivos_medicacao.questao_remedio_ambos)   
    ]
)

fig.update_layout(
    title='<b>Quantidade de infectados automedicados e medicados com orientacao médica</b>',
    legend_title='Legenda',
    width=800,
    height=600
)

fig.data[0].marker.color = ['#67000d',] * 2
fig.data[1].marker.color = ['#A50F15',] * 2 
fig.data[2].marker.color = ['#FB6A4A',] * 2 

fig.show()

**Qual é a porcentagem de internados que precisaram de respiração mecânica?**

In [14]:
df.show(3)

+----------+---------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|      data|       uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|questao_internacao_ajud

In [15]:
df_testes_positivos_internacao = spark.sql(

'''
    select
        count(resultado_teste) AS qtd_testes_positivos,
        CASE
        WHEN questao_internacao = 'Sim' THEN 'Internado'
        WHEN questao_internacao = 'Não' THEN 'Não internado'
        WHEN questao_internacao = 'Não foi atendido' THEN 'Não foi atendido'
        WHEN questao_internacao = 'Ignorado' THEN 'Ignorado'
        ELSE 'Não aplicável' 
        END AS questao_internacao
    FROM df_temp
    WHERE resultado_teste = 'Positivo' AND (questao_internacao = 'Sim' OR questao_internacao = 'Não')
    GROUP BY questao_internacao
'''
).toPandas()

fig = px.pie(
    data_frame=df_testes_positivos_internacao.sort_values('qtd_testes_positivos', ascending=False),
    values='qtd_testes_positivos',
    names='questao_internacao',
    color_discrete_sequence=px.colors.sequential.Reds_r,
    labels={
        'qtd_testes_positivos':'Quantidade',
        'questao_internacao':'Condição'
    }
)

fig.update_traces(
    pull=[0, 0.1],
    texttemplate='%{percent:.2%}'
)

fig.update_layout(
    title='<b>Porcentagem de infectados internados</b>',
    legend_title='Legenda',
    width=800,
    height=600
)

fig.show()